# Immuno Drug Repurposing - DeepChem Multitask Classification

In [0]:
%tensorflow_version 1.x
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)
import deepchem as dc

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import pandas as pd

# define the path to the dataset for training
input_data=os.path.join("/content/drive/My Drive/MyProjects/DeepChem/multiclass/datasets",'multiclass_origDS_noDB.csv')

# define the path to new prediction data
pred_data=os.path.join("/content/drive/My Drive/MyProjects/DeepChem/multiclass/datasets",'DB_toPredict.csv')

In [0]:
# read data
df_orig = pd.read_csv(input_data)

# get the names of tasks
tasksx = list(set(df_orig['UNIPROT']))

In [0]:
tasks = ['O00571', 'P00533', 'P01024', 'P01130', 'P04233', 'P07339', 'P08962', 'P09668',
         'P11021', 'P15291', 'P16070', 'P17301', 'P21741', 'P25774', 'P25963', 'P26006',
         'P27361', 'P35222', 'P40763', 'P50591', 'P55085', 'Q15904', 'Q16665', 'Q99519', 'Q99814']
len(tasks)

25

In [0]:
print(tasks)

['O00571', 'P00533', 'P01024', 'P01130', 'P04233', 'P07339', 'P08962', 'P09668', 'P11021', 'P15291', 'P16070', 'P17301', 'P21741', 'P25774', 'P25963', 'P26006', 'P27361', 'P35222', 'P40763', 'P50591', 'P55085', 'Q15904', 'Q16665', 'Q99519', 'Q99814']


In [0]:
# define features to calculate from SMILES as Circular Fingerprints (vectors of 1024 values)
featurizer=dc.feat.CircularFingerprint(size=1024) # dc.feat.ConvMolFeaturizer()

# load data and calculate the features for dataset
loader = dc.data.CSVLoader(tasks=tasks, smiles_field="canonical_smiles",featurizer=featurizer)
dataset=loader.featurize(input_data,data_dir='/content/drive/My Drive/MyProjects/DeepChem/multiclass/features_CFP_multiclass/')

# calculate the same features for new data to predict
loader2 = dc.data.CSVLoader(tasks=tasks, smiles_field="smiles",featurizer=featurizer)
dataset_pred=loader2.featurize(pred_data, data_dir='/content/drive/My Drive/MyProjects/DeepChem/multiclass/features_CFP_DB/')

Loading raw samples now.
shard_size: 8192
About to start loading CSV from /content/drive/My Drive/MyProjects/DeepChem/multiclass/datasets/multiclass_origDS_noDB.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 28.063 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
TIMING: featurizing shard 1 took 24.476 s
TIMING: dataset construction took 53.136 s
Loading dataset from disk.
Loading raw samples now.
shard_size: 8192
About to start loading CSV from /content/drive/My Drive/MyProjects/DeepChem/multiclass/datasets/DB_toPredict.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing s

In [0]:
print('Full dataset samples : {}'.format(dataset.X.shape[0]))
print('External dataset samples : {}'.format(dataset_pred.X.shape[0]))

Full dataset samples : 15377
External dataset samples : 10668


In [0]:
dataset.X.shape, dataset.y.shape

((15377, 1024), (15377, 25))

In [0]:
dataset_pred.X.shape, dataset_pred.y.shape

((10668, 1024), (10668, 25))

### Split dataset

In [0]:
splitter = dc.splits.RandomStratifiedSplitter(input_data)
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
    dataset, seed=42)

TIMING: dataset construction took 0.410 s
Loading dataset from disk.
TIMING: dataset construction took 0.460 s
Loading dataset from disk.
TIMING: dataset construction took 0.133 s
Loading dataset from disk.
TIMING: dataset construction took 0.473 s
Loading dataset from disk.


In [0]:
# create a list of parameters for a grid search
params_dict = {"activation": ["relu"],
               "momentum": [.9],
               "batch_size": [128, 515],
               "init": ["glorot_uniform"],
               "data_shape": [train_dataset.get_data_shape()],
               "learning_rate": [1e-3],
               "decay": [1e-6],
               "nb_epoch": [10],
               "nesterov": [False],
               "dropouts": [(0.0), (0.1), (0.2), (0.3)],
               "nb_layers": [1],
               "batchnorm": [False, True],
               "layer_sizes": [(100,), (500,), (1000,), (1024,)],
               "weight_init_stddevs": [(.1,)],
               "bias_init_consts": [(1.,)],
               "penalty": [0.], 
              } 

n_features = train_dataset.get_data_shape()[0]

# define a model builder
def model_builder(model_params, model_dir):
  model = dc.models.MultitaskClassifier(
    len(tasks), n_features, **model_params)
  return model

# set the metrics as mean AUROC
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

# grid search for the best model
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_dnn, best_hyperparams, all_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, [], metric)

Fitting model 1/64
hyperparameters: {'activation': 'relu', 'momentum': 0.9, 'batch_size': 128, 'init': 'glorot_uniform', 'data_shape': (1024,), 'learning_rate': 0.001, 'decay': 1e-06, 'nb_epoch': 10, 'nesterov': False, 'dropouts': 0.0, 'nb_layers': 1, 'batchnorm': False, 'layer_sizes': (100,), 'weight_init_stddevs': (0.1,), 'bias_init_consts': (1.0,), 'penalty': 0.0}


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))
/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [1.0, 0.9980861244019138, 0.9991237882177479, 0.9997554148436014, 1.0, 1.0, 1.0, 0.9347506132461161, 0.8997848394571335, 0.985552366232428, 0.9527455121436115, 1.0, 1.0, 0.9341715976331361, 1.0, 0.9989300672227501, 0.9998852750530602, 1.0, nan, 0.9927818030949795, nan, 1.0, 1.0, 0.9304399524375744, 1.0]
Model 1/64, Metric mean-roc_auc_score, Validation set 0: 0.983739
	best_validation_score so far: 0.983739
Fitting model 2/64
hyperparameters: {'activation': 'relu', 'momentum': 0.9, 'batch_size': 128, 'init': 'glorot_uniform', 'data_shape': (1024,), 'learning_rate': 0.001, 'decay': 1e-06, 'nb_epoch': 10, 'nesterov': False, 'dropouts': 0.0, 'nb_layers': 1, 'batchnorm': False, 'layer_sizes': (500,), 'weight_init_stddevs': (0.1,), 'bias_init_consts': (1.0,), 'penalty': 0.0}


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))
/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [1.0, 0.9980861244019139, 0.9990026099925429, 0.9999728238715113, 1.0, 1.0, 1.0, 0.9524529844644317, 0.8962264150943396, 0.9845219695296976, 0.8662222808870116, 1.0, 1.0, 0.9331854043392507, 1.0, 0.9989723014113259, 1.0, 1.0, nan, 0.9929101189916081, nan, 1.0, 1.0, 0.9250320131711333, 1.0]
Model 2/64, Metric mean-roc_auc_score, Validation set 1: 0.980286
	best_validation_score so far: 0.983739
Fitting model 3/64
hyperparameters: {'activation': 'relu', 'momentum': 0.9, 'batch_size': 128, 'init': 'glorot_uniform', 'data_shape': (1024,), 'learning_rate': 0.001, 'decay': 1e-06, 'nb_epoch': 10, 'nesterov': False, 'dropouts': 0.0, 'nb_layers': 1, 'batchnorm': False, 'layer_sizes': (1000,), 'weight_init_stddevs': (0.1,), 'bias_init_consts': (1.0,), 'penalty': 0.0}


...

computed_metrics: [1.0, 0.9980861244019139, 0.9992977877206064, 0.9998369432290676, 1.0, 1.0, 0.998769987699877, 0.9513491414554375, 0.886433851925411, 0.9872598807683814, 0.8183738120380148, 1.0, 1.0, 0.9820019723865878, 1.0, 0.9990321331784746, 0.9997131876326508, 1.0, nan, 0.9918613503964961, nan, 0.9998075442648191, 1.0, 0.9363852556480381, 1.0]
Model 63/64, Metric mean-roc_auc_score, Validation set 62: 0.980357
	best_validation_score so far: 0.988898
Fitting model 64/64
hyperparameters: {'activation': 'relu', 'momentum': 0.9, 'batch_size': 515, 'init': 'glorot_uniform', 'data_shape': (1024,), 'learning_rate': 0.001, 'decay': 1e-06, 'nb_epoch': 10, 'nesterov': False, 'dropouts': 0.3, 'nb_layers': 1, 'batchnorm': True, 'layer_sizes': (1024,), 'weight_init_stddevs': (0.1,), 'bias_init_consts': (1.0,), 'penalty': 0.0}


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))
/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [1.0, 0.9980861244019139, 0.9993878946060155, 0.999891295486045, 1.0, 1.0, 0.998769987699877, 0.9434587080948488, 0.8803100518592077, 0.9872230808861411, 0.8502507919746568, 1.0, 1.0, 0.8909023668639053, 1.0, 0.9990462112413332, 0.9997705501061207, 1.0, nan, 0.9925833411748604, nan, 1.0, 1.0, 0.931629013079667, 1.0]
Model 64/64, Metric mean-roc_auc_score, Validation set 63: 0.977013
	best_validation_score so far: 0.988898


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [1.0, 0.9994712176509621, 0.999965271638926, 0.9995932522042155, 1.0, 1.0, 0.9999803713736113, 0.9991580777127096, 0.999062765822322, 0.9984050580532093, 1.0, 0.9999881887438729, 1.0, 0.999891175579249, 1.0, 0.999846325525338, 0.9999917161150873, 1.0, nan, 0.9982996780331923, 0.9998212157330155, 0.9999141557215212, 1.0, 0.9995659884846442, 1.0]
Best hyperparameters: ('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.2, 1, False, (100,), (0.1,), (1.0,), 0.0)
train_score: 0.999706
validation_score: 0.988898


In [0]:
best_dnn

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


MultitaskClassifier(activation_fns=None, bias_init_consts=None, dropouts=None,
                    layer_sizes=None, n_classes=2, n_features=1024, n_tasks=25,
                    weight_decay_penalty=None, weight_decay_penalty_type=None,
                    weight_init_stddevs=None)

In [0]:
all_results

{"('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, False, (100,), (0.1,), (1.0,), 0.0)": 0.9837394501732196,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, False, (1000,), (0.1,), (1.0,), 0.0)": 0.9845269048460475,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, False, (1024,), (0.1,), (1.0,), 0.0)": 0.9844438998909272,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, False, (500,), (0.1,), (1.0,), 0.0)": 0.9802863063545552,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, True, (100,), (0.1,), (1.0,), 0.0)": 0.9846666190829115,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, True, (1000,), (0.1,), (1.0,), 0.0)": 0.9855107598216795,
 "('relu', 0.9, 128, 'glorot_uniform', (1024,), 0.001, 1e-06, 10, False, 0.0, 1, True, (1024,), (0.1,), (1.0,), 0.0)": 0.9842769459910852,
 "('relu', 0.9, 128, 'glor

### Use / Try the best configuration

In [0]:
params_dict = {"activation": ["relu"],
               "momentum": [.9],
               "batch_size": [124],
               "init": ["glorot_uniform"],
               "data_shape": [train_dataset.get_data_shape()],
               "learning_rate": [1e-3],
               "decay": [1e-6],
               "nb_epoch": [1],
               "nesterov": [False],
               "dropouts": [(0.5,)],
               "nb_layers": [1],
               "batchnorm": [False],
               "layer_sizes": [(1000,)],
               "weight_init_stddevs": [(.1,)],
               "bias_init_consts": [(1.,)],
               "penalty": [0.], 
              } 

n_features = train_dataset.get_data_shape()[0]
def model_builder(model_params, model_dir):
  model = dc.models.MultitaskClassifier(
    len(tasks), n_features, **model_params)
  return model

metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_dnn, best_hyperparams, all_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, [], metric)

Fitting model 1/1
hyperparameters: {'activation': 'relu', 'momentum': 0.9, 'batch_size': 124, 'init': 'glorot_uniform', 'data_shape': (1024,), 'learning_rate': 0.001, 'decay': 1e-06, 'nb_epoch': 1, 'nesterov': False, 'dropouts': (0.5,), 'nb_layers': 1, 'batchnorm': False, 'layer_sizes': (1000,), 'weight_init_stddevs': (0.1,), 'bias_init_consts': (1.0,), 'penalty': 0.0}


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))
/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


computed_metrics: [1.0, 0.993931879341636, 1.0, 1.0, 1.0, 0.9995023338824822, 1.0, 0.9940180730558738, nan, 0.9983660130718954, 1.0, nan, 1.0, 0.9989606190091236, 1.0, 1.0, 0.9891141732283464, 0.9295833333333333, 0.9803770753212644, 1.0, 0.9991474850809889, 0.06064296151972724, 0.9856998060990657, 1.0, 1.0]
Model 1/1, Metric mean-roc_auc_score, Validation set 0: 0.953450
	best_validation_score so far: 0.953450
computed_metrics: [1.0, 0.9996313199872222, 1.0, 1.0, 0.9993603581994084, 0.9999845274081491, 1.0, 0.9986923429299922, nan, 1.0, 1.0, 1.0, 1.0, 0.9998870004302973, 1.0, 1.0, 0.9995973356006437, 1.0, 0.9999552191737622, 1.0, 1.0, 1.0, 0.9999993303699974, 1.0, 0.999999436281805]
Best hyperparameters: ('relu', 0.9, 124, 'glorot_uniform', (1024,), 0.001, 1e-06, 1, False, (0.5,), 1, False, (1000,), (0.1,), (1.0,), 0.0)
train_score: 0.999879
validation_score: 0.953450


/usr/local/lib/python3.7/site-packages/deepchem/metrics/__init__.py:368: UserWarning: Error calculating metric mean-roc_auc_score: Found input variables with inconsistent numbers of samples: [1, 2]
  warnings.warn("Error calculating metric %s: %s" % (self.name, e))


### Evaluate the best model using the test subset

In [0]:
metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

In [0]:
test_scores = best_dnn.evaluate(test_dataset, [metric])
print("Test ROC-AUC Score: %f" % test_scores["mean-roc_auc_score"])

computed_metrics: [0.9846491228070176, 0.9929062338173177, 1.0, 1.0, 0.9977560179518563, 0.9993239828152641, 1.0, 0.9913397129186603, 0.3555027969298816, 1.0, 1.0, 1.0, 1.0, 0.9992952428895041, 0.9523353084698502, 1.0, 0.9456159420289856, 0.9352351485148515, 0.9851405519953021, 1.0, 0.9999630860095977, 1.0, 0.9718115292712068, 1.0, 1.0]
Test ROC-AUC Score: 0.964435


Conclusion: we obtained a multitask model for immuno drug repurposing with mean AUROC of 0.964.

### Get the metris as file

In [0]:
df_scores = pd.DataFrame(columns=['UNIPROT', 'AUROC train', 'AUROC valid', 'AUROC test'])
df_scores['UNIPROT'] = tasks
df_scores['AUROC train'] = [1.0, 0.993931879341636, 1.0, 1.0, 1.0, 0.9995023338824822, 1.0, 0.9940180730558738, None, 0.9983660130718954, 1.0, None, 1.0,
                            0.9989606190091236, 1.0, 1.0, 0.9891141732283464, 0.9295833333333333, 0.9803770753212644, 1.0, 0.9991474850809889, 0.06064296151972724, 0.9856998060990657, 1.0, 1.0]
df_scores['AUROC valid'] = [1.0, 0.9996313199872222, 1.0, 1.0, 0.9993603581994084, 0.9999845274081491, 1.0, 0.9986923429299922, None, 1.0, 1.0, 1.0, 1.0,
                            0.9998870004302973, 1.0, 1.0, 0.9995973356006437, 1.0, 0.9999552191737622, 1.0, 1.0, 1.0, 0.9999993303699974, 1.0, 0.999999436281805]
df_scores['AUROC test'] = [0.9846491228070176, 0.9929062338173177, 1.0, 1.0, 0.9977560179518563, 0.9993239828152641, 1.0, 0.9913397129186603, 0.3555027969298816, 1.0, 1.0, 1.0, 1.0,
                           0.9992952428895041, 0.9523353084698502, 1.0, 0.9456159420289856, 0.9352351485148515, 0.9851405519953021, 1.0, 0.9999630860095977, 1.0, 0.9718115292712068, 1.0, 1.0]

df_scores.to_csv('/content/drive/My Drive/MyProjects/DeepChem/multiclass/results/multitasks_metrics_best.csv',index=False) 

### Make predictions for Databank drugs with the best model

In [0]:
# use the calculated descriptors with the best model
predictions = best_dnn.predict(dataset_pred)

In [0]:
predictions.shape

(10668, 25, 2)

In [0]:
predictions[:,0,1].shape

(10668,)

In [0]:
predictions[:,:,1].shape

(10668, 25)

In [0]:
dataset_pred.ids

array(['CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O',
       'CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](CC(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=C1C=CC=C2)NC(=O)[C@H](CC1=CNC=N1)NC(=O)[C@@H]1CCC(=O)N1',
       'CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=CC=CC=C12)NC(=O)[C@H](CC1=CN=CN1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N1CCC[C@H]1C(=O)NNC(N)=O',
       ..., 'CN(C)CC(CN1C2=CC=CC=C2SC2=C1C=CC=C2)N(C)C',
       'CC(CC(C(N)=O)(C1=CC=CC=C1)C1=CC=CC=C1)N(C)C',
       '[Fe+3].CC1=C([O-])C(=O)C=CO1.CC1=C([O-])C(=O)C=CO1.CC1=C([O-])C(=O)C=CO1'],
      

Add more columns to the predictions:

In [0]:
df_new = pd.DataFrame(predictions[:,:,1], columns=tasks)
df_new

,O00571,P00533,P01024,P01130,P04233,P07339,P08962,P09668,P11021,P15291,P16070,P17301,P21741,P25774,P25963,P26006,P27361,P35222,P40763,P50591,P55085,Q15904,Q16665,Q99519,Q99814
0,5.952556e-06,7.675065e-07,0.000002,5.780248e-05,6.573859e-07,0.000411,0.000008,0.000272,0.000038,0.000006,0.000010,0.000003,6.768991e-07,0.000051,0.000007,6.520727e-09,1.130938e-06,0.000035,0.000048,1.874270e-06,0.074258,2.097711e-07,0.891102,6.894741e-06,0.000006
1,3.072290e-07,1.839695e-04,0.000543,1.181113e-05,8.107681e-06,0.005194,0.000008,0.000373,0.000219,0.000030,0.000027,0.000067,1.835666e-06,0.021066,0.000003,2.184425e-08,1.935587e-03,0.000077,0.000003,7.596868e-07,0.000424,2.328674e-07,0.000404,5.779951e-07,0.000002
2,2.360307e-06,1.371730e-05,0.000128,3.388771e-05,1.962694e-05,0.000570,0.000002,0.000102,0.001474,0.000006,0.000051,0.000026,6.764506e-07,0.657537,0.000026,2.074363e-08,2.692508e-02,0.000494,0.001917,8.034566e-07,0.000075,1.321061e-06,0.000018,7.565588e-06,0.000029
3,2.052896e-05,4.322432e-07,0.000257,1.501892e-05,1.003678e-06,0.000876,0.000003,0.002826,0.000017,0.000257,0.000013,0.000121,3.063406e-06,0.123659,0.000004,2.681151e-07,6.684351e-05,0.001614,0.001198,5.883117e-06,0.003135,3.060142e-07,0.000303,2.964538e-06,0.000024
4,1.249870e-06,1.692883e-06,0.000303,3.987369e-06,5.112667e-05,0.000126,0.000030,0.000031,0.000052,0.000010,0.000055,0.000014,2.798275e-06,0.073906,0.000067,2.045474e-05,7.554174e-06,0.000183,0.000003,3.983538e-06,0.000027,4.026024e-07,0.000683,3.265450e-06,0.000066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10663,1.210241e-05,7.647240e-01,0.000001,5.401958e-05,1.944366e-06,0.077596,0.000001,0.000004,0.000013,0.000004,0.000004,0.000021,1.461617e-05,0.000004,0.000011,4.591110e-08,7.602058e-07,0.000875,0.000022,1.349965e-05,0.000222,2.271440e-06,0.116935,1.260030e-04,0.001210
10664,2.620578e-05,1.524297e-03,0.000001,5.111931e-05,8.821132e-06,0.000144,0.000008,0.000005,0.000045,0.000013,0.000012,0.000004,1.349787e-05,0.000116,0.000088,2.010841e-07,2.186017e-05,0.006708,0.014000,4.085128e-06,0.000163,1.889475e-07,0.002980,1.977859e-05,0.001873
10665,6.037804e-05,9.919590e-01,0.000002,4.480966e-07,3.029137e-05,0.000006,0.000008,0.000359,0.000009,0.000115,0.000034,0.000092,6.157114e-06,0.005918,0.000006,6.246655e-07,1.671474e-05,0.008014,0.000471,9.150760e-05,0.000008,8.585966e-06,0.006676,3.075923e-04,0.000064
10666,7.689733e-05,9.455895e-01,0.000005,1.332135e-05,6.231962e-06,0.000509,0.000037,0.002065,0.000027,0.000097,0.000043,0.000097,1.028937e-05,0.000070,0.000014,3.701699e-07,1.723128e-05,0.000858,0.051007,1.048607e-04,0.000590,2.109816e-06,0.026305,2.875570e-04,0.000003


In [0]:
df_new['smiles'] = list(dataset_pred.ids)
df_new

,O00571,P00533,P01024,P01130,P04233,P07339,P08962,P09668,P11021,P15291,P16070,P17301,P21741,P25774,P25963,P26006,P27361,P35222,P40763,P50591,P55085,Q15904,Q16665,Q99519,Q99814,smiles
0,5.952556e-06,7.675065e-07,0.000002,5.780248e-05,6.573859e-07,0.000411,0.000008,0.000272,0.000038,0.000006,0.000010,0.000003,6.768991e-07,0.000051,0.000007,6.520727e-09,1.130938e-06,0.000035,0.000048,1.874270e-06,0.074258,2.097711e-07,0.891102,6.894741e-06,0.000006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,3.072290e-07,1.839695e-04,0.000543,1.181113e-05,8.107681e-06,0.005194,0.000008,0.000373,0.000219,0.000030,0.000027,0.000067,1.835666e-06,0.021066,0.000003,2.184425e-08,1.935587e-03,0.000077,0.000003,7.596868e-07,0.000424,2.328674e-07,0.000404,5.779951e-07,0.000002,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...
2,2.360307e-06,1.371730e-05,0.000128,3.388771e-05,1.962694e-05,0.000570,0.000002,0.000102,0.001474,0.000006,0.000051,0.000026,6.764506e-07,0.657537,0.000026,2.074363e-08,2.692508e-02,0.000494,0.001917,8.034566e-07,0.000075,1.321061e-06,0.000018,7.565588e-06,0.000029,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
3,2.052896e-05,4.322432e-07,0.000257,1.501892e-05,1.003678e-06,0.000876,0.000003,0.002826,0.000017,0.000257,0.000013,0.000121,3.063406e-06,0.123659,0.000004,2.681151e-07,6.684351e-05,0.001614,0.001198,5.883117e-06,0.003135,3.060142e-07,0.000303,2.964538e-06,0.000024,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
4,1.249870e-06,1.692883e-06,0.000303,3.987369e-06,5.112667e-05,0.000126,0.000030,0.000031,0.000052,0.000010,0.000055,0.000014,2.798275e-06,0.073906,0.000067,2.045474e-05,7.554174e-06,0.000183,0.000003,3.983538e-06,0.000027,4.026024e-07,0.000683,3.265450e-06,0.000066,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10663,1.210241e-05,7.647240e-01,0.000001,5.401958e-05,1.944366e-06,0.077596,0.000001,0.000004,0.000013,0.000004,0.000004,0.000021,1.461617e-05,0.000004,0.000011,4.591110e-08,7.602058e-07,0.000875,0.000022,1.349965e-05,0.000222,2.271440e-06,0.116935,1.260030e-04,0.001210,COC1=CC(=CC(OC)=C1OC)[C@H]1C2=C(COC2=O)CC2=C1C...
10664,2.620578e-05,1.524297e-03,0.000001,5.111931e-05,8.821132e-06,0.000144,0.000008,0.000005,0.000045,0.000013,0.000012,0.000004,1.349787e-05,0.000116,0.000088,2.010841e-07,2.186017e-05,0.006708,0.014000,4.085128e-06,0.000163,1.889475e-07,0.002980,1.977859e-05,0.001873,[Na+].CCOC1=CC=C(NC2=CC=C(C=C2)C(=C2\C=C/C(/C=...
10665,6.037804e-05,9.919590e-01,0.000002,4.480966e-07,3.029137e-05,0.000006,0.000008,0.000359,0.000009,0.000115,0.000034,0.000092,6.157114e-06,0.005918,0.000006,6.246655e-07,1.671474e-05,0.008014,0.000471,9.150760e-05,0.000008,8.585966e-06,0.006676,3.075923e-04,0.000064,CN(C)CC(CN1C2=CC=CC=C2SC2=C1C=CC=C2)N(C)C
10666,7.689733e-05,9.455895e-01,0.000005,1.332135e-05,6.231962e-06,0.000509,0.000037,0.002065,0.000027,0.000097,0.000043,0.000097,1.028937e-05,0.000070,0.000014,3.701699e-07,1.723128e-05,0.000858,0.051007,1.048607e-04,0.000590,2.109816e-06,0.026305,2.875570e-04,0.000003,CC(CC(C(N)=O)(C1=CC=CC=C1)C1=CC=CC=C1)N(C)C


### Add DB ids

In [0]:
# read data to predict for DB ids
df_x = pd.read_csv('/content/drive/My Drive/MyProjects/DeepChem/multiclass/datasets/DB_toPredict.csv')
df_x.columns

Index(['DB_id', 'smiles', 'FILTEREDCLASS', 'O00571', 'P00533', 'P01024',
       'P01130', 'P04233', 'P07339', 'P08962', 'P09668', 'P11021', 'P15291',
       'P16070', 'P17301', 'P21741', 'P25774', 'P25963', 'P26006', 'P27361',
       'P35222', 'P40763', 'P50591', 'P55085', 'Q15904', 'Q16665', 'Q99519',
       'Q99814'],
      dtype='object')

In [0]:
df_x.shape

(10691, 28)

In [0]:
df_new.columns

Index(['O00571', 'P00533', 'P01024', 'P01130', 'P04233', 'P07339', 'P08962',
       'P09668', 'P11021', 'P15291', 'P16070', 'P17301', 'P21741', 'P25774',
       'P25963', 'P26006', 'P27361', 'P35222', 'P40763', 'P50591', 'P55085',
       'Q15904', 'Q16665', 'Q99519', 'Q99814', 'smiles'],
      dtype='object')

In [0]:
merged_inner = pd.merge(left=df_x[['DB_id', 'smiles']], right=df_new, on='smiles', how='inner')
merged_inner

,DB_id,smiles,O00571,P00533,P01024,P01130,P04233,P07339,P08962,P09668,P11021,P15291,P16070,P17301,P21741,P25774,P25963,P26006,P27361,P35222,P40763,P50591,P55085,Q15904,Q16665,Q99519,Q99814
0,DB00006,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,5.952556e-06,7.675065e-07,0.000002,5.780248e-05,6.573859e-07,0.000411,0.000008,0.000272,0.000038,0.000006,0.000010,0.000003,6.768991e-07,0.000051,0.000007,6.520727e-09,1.130938e-06,0.000035,0.000048,1.874270e-06,0.074258,2.097711e-07,0.891102,6.894741e-06,0.000006
1,DB00007,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,3.072290e-07,1.839695e-04,0.000543,1.181113e-05,8.107681e-06,0.005194,0.000008,0.000373,0.000219,0.000030,0.000027,0.000067,1.835666e-06,0.021066,0.000003,2.184425e-08,1.935587e-03,0.000077,0.000003,7.596868e-07,0.000424,2.328674e-07,0.000404,5.779951e-07,0.000002
2,DB00014,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,2.360307e-06,1.371730e-05,0.000128,3.388771e-05,1.962694e-05,0.000570,0.000002,0.000102,0.001474,0.000006,0.000051,0.000026,6.764506e-07,0.657537,0.000026,2.074363e-08,2.692508e-02,0.000494,0.001917,8.034566e-07,0.000075,1.321061e-06,0.000018,7.565588e-06,0.000029
3,DB00027,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,2.052896e-05,4.322432e-07,0.000257,1.501892e-05,1.003678e-06,0.000876,0.000003,0.002826,0.000017,0.000257,0.000013,0.000121,3.063406e-06,0.123659,0.000004,2.681151e-07,6.684351e-05,0.001614,0.001198,5.883117e-06,0.003135,3.060142e-07,0.000303,2.964538e-06,0.000024
4,DB00035,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,1.249870e-06,1.692883e-06,0.000303,3.987369e-06,5.112667e-05,0.000126,0.000030,0.000031,0.000052,0.000010,0.000055,0.000014,2.798275e-06,0.073906,0.000067,2.045474e-05,7.554174e-06,0.000183,0.000003,3.983538e-06,0.000027,4.026024e-07,0.000683,3.265450e-06,0.000066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10667,DB15590,COC1=CC(=CC(OC)=C1OC)[C@H]1C2=C(COC2=O)CC2=C1C...,1.210241e-05,7.647240e-01,0.000001,5.401958e-05,1.944366e-06,0.077596,0.000001,0.000004,0.000013,0.000004,0.000004,0.000021,1.461617e-05,0.000004,0.000011,4.591110e-08,7.602058e-07,0.000875,0.000022,1.349965e-05,0.000222,2.271440e-06,0.116935,1.260030e-04,0.001210
10668,DB15594,[Na+].CCOC1=CC=C(NC2=CC=C(C=C2)C(=C2\C=C/C(/C=...,2.620578e-05,1.524297e-03,0.000001,5.111931e-05,8.821132e-06,0.000144,0.000008,0.000005,0.000045,0.000013,0.000012,0.000004,1.349787e-05,0.000116,0.000088,2.010841e-07,2.186017e-05,0.006708,0.014000,4.085128e-06,0.000163,1.889475e-07,0.002980,1.977859e-05,0.001873
10669,DB15596,CN(C)CC(CN1C2=CC=CC=C2SC2=C1C=CC=C2)N(C)C,6.037804e-05,9.919590e-01,0.000002,4.480966e-07,3.029137e-05,0.000006,0.000008,0.000359,0.000009,0.000115,0.000034,0.000092,6.157114e-06,0.005918,0.000006,6.246655e-07,1.671474e-05,0.008014,0.000471,9.150760e-05,0.000008,8.585966e-06,0.006676,3.075923e-04,0.000064
10670,DB15597,CC(CC(C(N)=O)(C1=CC=CC=C1)C1=CC=CC=C1)N(C)C,7.689733e-05,9.455895e-01,0.000005,1.332135e-05,6.231962e-06,0.000509,0.000037,0.002065,0.000027,0.000097,0.000043,0.000097,1.028937e-05,0.000070,0.000014,3.701699e-07,1.723128e-05,0.000858,0.051007,1.048607e-04,0.000590,2.109816e-06,0.026305,2.875570e-04,0.000003


In [0]:
# save the predictions to file
merged_inner.to_csv('/content/drive/My Drive/MyProjects/DeepChem/multiclass/results/multitasks_predictions_best.csv', index=False)